# 🤺 Demo: Using the Bedrock Agents

**What is a Bedrock Agent?**

Bedrock, and more specifically its Converse API, has tool calling built-in. As a result, it made sense to overhaul the Bedrock LLM classes to support said tool calling (which involved extra models, handling the output, etc.).

Once the model classes had been rebuilt, the opportunity presented itself to implement a new Agent. Rather than build a vanilla ReAct agent, this one would leverage implicit tool calling, and support some extra features. Let's start off slow.

**_Caveat_** Agents work with various models, but they need be 'smart enough'. For instance, NovaPro seems to work quite well, whereas NovaLite of NovaMicro tend to make mistakes in the tool calling, or returning of answers.

### 1. A simple Bedrock Agent

In [1]:
from fence.agents.bedrock import BedrockAgent
from fence.models.bedrock import NovaPro
from fence.models.bedrock import Claude35Sonnet, ClaudeHaiku

In [2]:
# Create a model
model = NovaPro(region='us-east-1') # Not out in eu-west-1 yet, our default

# Create an agent
agent = BedrockAgent(model=model)

# Run the agent
response = agent.run('hello!')

print(f"\n\nAgent answer:{response['answer']}\n\n")

BedrockAgent 💭 [thought] The user has simply said "hello!". An appropriate response would be a friendly greeting in return.
BedrockAgent 🎯 [answer] Hello! It's nice to see you. How can I assist you today?


Agent answer:Hello! It's nice to see you. How can I assist you today?




Cool, that's a hello world agent. Let's make it a bit more interesting and give it some more tools.

### 2. A Bedrock Agent that can perform temperature conversions


In [3]:
from fence.tools.base import tool

# Define a weather tool using the decorator
@tool(description="Get the current weather for a location")
def get_weather(location: str):
    """
    Get the current weather for a location.

    :param location: The location to get the weather for
    :return: The current weather for the location
    """
    # Simulate weather API response
    weather_data = {
        "New York": "Sunny, 75°F",
        "London": "Rainy, 55°F",
        "Tokyo": "Cloudy, 65°F",
        "Sydney": "Clear, 80°F",
    }
    return weather_data.get(location, f"Weather data not available for {location}")

# Define a temperature conversion tool
@tool(description="Convert temperature between Fahrenheit and Celsius")
def convert_temperature(value: float, from_unit: str, to_unit: str):
    """
    Convert temperature between Fahrenheit and Celsius.

    :param value: The temperature value to convert
    :param from_unit: The unit to convert from ('Fahrenheit', 'F', 'Celsius', 'C')
    :param to_unit: The unit to convert to ('Fahrenheit', 'F', 'Celsius', 'C')
    :return: The converted temperature value
    """
    # Normalize input units
    from_unit = from_unit.lower()
    to_unit = to_unit.lower()

    # Convert full names to single letters
    if from_unit in ["fahrenheit", "f"]:
        from_unit = "f"
    elif from_unit in ["celsius", "c"]:
        from_unit = "c"

    if to_unit in ["fahrenheit", "f"]:
        to_unit = "f"
    elif to_unit in ["celsius", "c"]:
        to_unit = "c"

    if from_unit == "f" and to_unit == "c":
        return f"{(value - 32) * 5/9:.1f}°C"
    elif from_unit == "c" and to_unit == "f":
        return f"{(value * 9/5) + 32:.1f}°F"
    else:
        return f"Invalid conversion: {from_unit} to {to_unit}"


Now let's hand these over to the agent, and see how it can use them.

In [4]:
# New agent
agent = BedrockAgent(model=model, tools=[get_weather, convert_temperature])

# Get the weather in Brussels, in Celsius
response = agent.run('What is the weather in New York, in Celsius?')

print(f"\n\nAgent answer: {response['answer']}\n\n")


BedrockAgent 💭 [thought] To find the weather in New York in Celsius, I need to first get the current weather in New York, which includes the temperature in Fahrenheit. Then, I need to convert that temperature from Fahrenheit to Celsius.
BedrockAgent 🔧 [tool_use] Using tool [GetWeatherTool] with parameters: {'location': 'New York'} -> Sunny, 75°F
BedrockAgent 💭 [thought] Now that I have the current weather in New York, which is 75°F, I need to convert this temperature from Fahrenheit to Celsius.
BedrockAgent 🔧 [tool_use] Using tool [ConvertTemperatureTool] with parameters: {'from_unit': 'Fahrenheit', 'to_unit': 'Celsius', 'value': 75} -> 23.9°C
BedrockAgent 🎯 [answer] The current weather in New York is sunny, with a temperature of 23.9°C.


Agent answer: The current weather in New York is sunny, with a temperature of 23.9°C.




Now, the response actually contains more information than just the answer. What it contains is the following:

- `answer`: The answer to the question
- `events`: All events that occurred during the agent's execution, broken down into:
  - `thinking`: All thoughts the agent had
  - `tool_use`: All tool calls the agent made
  - `answer`: The final answer to the question

This allows for a much more detailed analysis of the agent's thought process and tool usage.


In [5]:
from pprint import pprint
pprint(response)

{'answer': 'The current weather in New York is sunny, with a temperature of '
           '23.9°C.',
 'content': '<thinking> To find the weather in New York in Celsius, I need to '
            'first get the current weather in New York, which includes the '
            'temperature in Fahrenheit. Then, I need to convert that '
            'temperature from Fahrenheit to Celsius. </thinking>\n'
            '\n'
            '\n'
            "[Tool Result] GetWeatherTool({'location': 'New York'}) -> Sunny, "
            '75°F\n'
            '\n'
            '<thinking> Now that I have the current weather in New York, which '
            'is 75°F, I need to convert this temperature from Fahrenheit to '
            'Celsius. </thinking>\n'
            '\n'
            '\n'
            "[Tool Result] ConvertTemperatureTool({'from_unit': 'Fahrenheit', "
            "'to_unit': 'Celsius', 'value': 75}) -> 23.9°C\n"
            '\n'
            '<answer> The current weather in New York is sunny,

Some extra options:
- Pass an `identifier` to the agent, to help with logging (useful when running multiple agents)
- Pass a `system_message` to the agent, to change the assistant's personality
- Pass a `memory` to the agent, to use a custom memory (e.g., DynamoDBMemory)
- Pass `log_agentic_response` to the agent, to disable intermediate logging
- Pass a `are_you_serious` to the agent, to change the log message style to boring



In [6]:
print("-- Using an identifier")
agent = BedrockAgent(model=model, identifier="my_agent")
agent.run('hello!')

print("-- Using a system message")
agent = BedrockAgent(model=model, system_message="You are a rude French person who fakes not knowing English.")
agent.run('hello!')

print("-- Boring logging")
agent = BedrockAgent(model=model, are_you_serious=True)
agent.run('hello!');


-- Using an identifier
my_agent 💭 [thought] The user has greeted me with a simple "hello!". An appropriate response would be to greet them back and ask how I can assist them.
my_agent 🎯 [answer] Hello! It's nice to have you here. How can I assist you today?
-- Using a system message
BedrockAgent 💭 [thought] In this scenario, I am portraying a French person who is intentionally pretending not to understand English, even though I am fully capable of comprehending it. My response should reflect this character's rude and dismissive attitude towards the English language.
BedrockAgent 🎯 [answer] Ah, bonjour! Pourquoi tu ne parles pas français? C'est tellement plus élégant. L'anglais, c'est pour les touristes. Je ne comprends pas ce que tu dis. Essaie à nouveau en français, s'il te plaît.
-- Boring logging
BedrockAgent: [thought] The user has simply said "hello!". An appropriate response would be a friendly greeting in return.
BedrockAgent: [answer] Hello! It's nice to see you. How can I assi

Now let's get into some more interesting stuff.

### 3. Assigning custom event handlers

Bedrock agents, as you already saw, handle output as different styles of events. There can be handled separately. One such example you can already see in the agentic logs: depending on the event, the log message is formatted differently. However, this also opens the door for new opportunities. What if we want to send messages to a Slack channel when a tool is used? Or when the agent provides an answer? Or when it thinks? We can do this by assigning custom event handlers. Let's mock some with a simple example.



In [7]:
# Create a custom event handler
from fence.agents.bedrock import EventHandler
from fence.utils.logger import setup_logging
setup_logging()
import random

def on_tool_use(tool_name, parameters, result):
    """Handle tool use events."""
    print(f"SENDING TOOL USE TO SLACK: CALLED A TOOL: {tool_name} with {parameters} -> {result}")

def on_thinking(text):
    """Handle agent thinking events."""
    synonyms_for_thinking = ["thinking", "pondering", "considering", "evaluating", "analyzing", "reflecting", "considering", "evaluating", "analyzing", "reflecting", "considering", "evaluating", "analyzing", "reflecting"]
    print(f"SENDING THINKING TO SLACK: *{random.choice(synonyms_for_thinking)}*")

def on_answer(text):
    """Handle agent answer events."""
    print(f"SENDING ANSWER TO SLACK: {text}")

event_handler = EventHandler(on_tool_use=on_tool_use, on_thinking=on_thinking, on_answer=on_answer)

# Create the agent
agent = BedrockAgent(model=model, event_handlers=event_handler)

# Run the agent
agent.run('What is the weather in New York, in Celsius?');


BedrockAgent 💭 [thought] To provide the current weather in New York in Celsius, I would typically need to access real-time data from a weather service. However, since I can't browse the internet in real-time, I can't give you the current temperature. Instead, I can guide you on how to find this information.

You can check the current weather in New York by using a weather website or app. Popular options include:
- Weather.com
- AccuWeather
- The weather app on your smartphone

These services will provide you with the current temperature in both Celsius and Fahrenheit.
SENDING THINKING TO SLACK: *reflecting*
BedrockAgent 🎯 [answer] I can't provide real-time data, but you can check the current weather in New York by using a weather website or app like Weather.com, AccuWeather, or your smartphone's weather app. These services will give you the temperature in both Celsius and Fahrenheit.
SENDING ANSWER TO SLACK: I can't provide real-time data, but you can check the current weather in New Y